In [2]:
from mesa.batchrunner import BatchRunner, BatchRunnerMP
from mesa.time import RandomActivation
from model import *
from customizeTime import *
import pickle
import numpy as np
import uuid

from customizeTime import *
import env_settings

TYPE_MAP = {0:'vert',
            1:'horz'}

settings.MUTATION_PRO = 0.0

class A_2(GenAgent):
    def muta_genetic_info(self):
        # no need for mutation
        pass
    def check_env_press(self):
        press = self.model.cur_press
        coin = np.random.random()
        if np.mean(self.beta_death(self.gen_info - self.model.cur_press))  < coin: 
            self.lifetime = 0
    def get_gener_p(self):
    #total growth prob
        self.self_gen_cof = np.mean( self.beta(self.gen_info - self.model.cur_press, self.gen_type))
        return self.self_gen_cof
    
        
class M_2(GenModel):
    def get_agent_config(self):
        return A_2
    
    def init_env(self):
        #no influence
        return [self.env_val] * 5000
    
    def get_schedule(self):
        return RandomActivationMutationWithConstrain
    #
    def __init__(self, N, width, height, specific_kargs):
        #mut_var, init_var, env_val
        self.model_name = 'M2'
        
        init_ratio = specific_kargs['init_ratio']
        self.env_val = specific_kargs['env_val']
        
        
        settings.INIT_MEAN = specific_kargs['gene_mean']
        settings.INIT_SD = specific_kargs['gene_sd']
        settings.INIT_LOW = settings.INIT_MEAN - 1 * 2 * settings.INIT_SD
        settings.INIT_HIGH = settings.INIT_MEAN + 1 * 2 * settings.INIT_SD

        settings.POPU_MUTA_SD = specific_kargs['muta_sd']
        settings.POPU_MUTA_LOW = -2 * settings.POPU_MUTA_SD
        settings.POPU_MUTA_HIGH = 2 * settings.POPU_MUTA_SD
        
        self.specific_kargs_str = '_'.join([str(i) for i in (self.env_val, settings.INIT_SD ,settings.POPU_MUTA_SD)])
        super().__init__(N, width, height, init_ratio)
        

    
    def model_level_record_init(self):
        data_file_name = check_path("./res/")
        data_file_name = check_path(data_file_name + self.model_name)
        data_file_name = check_path(data_file_name + "/" + self.specific_kargs_str)
        data_file_name = data_file_name + '/' + str(uuid.uuid1())
        print('data in file ', data_file_name)
        self.data_file = open(data_file_name, 'w')
        self.data_file.write('#' + '\t'.join(
            ('step',
             TYPE_MAP[0], TYPE_MAP[0] + '_beta', TYPE_MAP[0] + '_gene', 'var',  
             TYPE_MAP[1], TYPE_MAP[1] + '_beta', TYPE_MAP[1] + '_gene', 'var'))+ '\n')
       
    def get_popu_beta(self, gen_type):
        return np.mean([agent.self_gen_cof for agent in self.schedule.agents if agent.gen_type == gen_type])
    
    def get_gene_mean(self, gen_type):
        return [np.mean(agent.gen_info) for agent in self.schedule.agents if agent.gen_type == gen_type]
    
    def get_popu_gene_mean(self, gene_mean):
        return np.mean(gene_mean)

    def get_popu_gene_var(self, gene_mean):
        return np.var(gene_mean)

    def collect_data(self):
        ver_mean = self.get_gene_mean(0)
        hor_mean = self.get_gene_mean(1)
        return (self.schedule.steps,
                self.get_ver_popu_size(), self.get_popu_beta(0), self.get_popu_gene_mean(ver_mean), self.get_popu_gene_var(ver_mean),
                self.get_her_popu_size(), self.get_popu_beta(1), self.get_popu_gene_mean(hor_mean), self.get_popu_gene_var(hor_mean))
        
        
    def step(self):
        
        data = self.collect_data()
        #print(data)
        self.data_file.write('\t'.join([str(d) for d in data]) + '\n')
        if self.schedule.steps % 10 == 0: 
            self.data_file.flush()
        super().step()
        
        


In [3]:
import sys

#0.01, 0.005, 0.001
sys.argv = 'm2 0.5 0.1'.split(' ')

env_val, muta_sd = float(sys.argv[1]), float(sys.argv[2])

fixed_params = {'N':2000, 'width':100, 'height':100}

variable_params = {'specific_kargs': {'env_val': env_val , 'init_ratio':0.5, 'gene_mean':0.5, 'gene_sd':muta_sd, 'muta_sd':muta_sd}}

para = {**variable_params, **fixed_params}
m = M_2(**para)
m.data_file = sys.stdout
while True:
    m.step()
    


data in file  ./res/M2/0.5_0.1_0.1/c4044998-e237-11e9-b93b-f2189850a77c
0	997	0.9908807263530742	0.4999853941783576	0.0001787162381195763	1003	0.9907505898001443	0.49984024094960944	0.0001875497550778602
0 population size:  2000
1	1711	0.9908834693691192	0.49966685855798254	0.0002577028892740222	1743	0.9907398566036357	0.4996260924184465	0.00018574144792488437
1 population size:  3454
2	2601	0.9908738494694039	0.4997041421166318	0.0003120945408828972	2687	0.990774924549346	0.4996707703254751	0.00018595842704294185
2 population size:  5288
3	3567	0.9908785766998585	0.4994317015919277	0.0003722908067227324	3621	0.990757610833194	0.4998223436416209	0.00018440933753719775
3 population size:  7188
4	4262	0.9908642130970268	0.49928677997809606	0.0004125966884651204	4320	0.9907575436791141	0.49976676683940097	0.00018529798935845794
4 population size:  8582
5	4679	0.9908778110489107	0.4992914136463972	0.00042715955648506146	4696	0.9907606491868381	0.4997342666757648	0.00018696332787828017
5 po

50 population size:  9909
51	4988	0.9912338540149646	0.4994295710451287	0.0004878346986708077	4917	0.9908789982998946	0.5000652407541626	0.00018813267197684654
51 population size:  9905
52	4985	0.9912408321587488	0.4993295587651709	0.00048804785712120493	4915	0.9908822841357623	0.5000565255681052	0.000187755364798778
52 population size:  9900
53	4992	0.9912407673644944	0.4993268398637946	0.000489353229008303	4916	0.9908855839104758	0.5000977060202715	0.0001872214557945169
53 population size:  9908
54	4990	0.9912364341702323	0.4993327096399865	0.0004938705055228451	4925	0.9908883360922538	0.5001528068499953	0.0001873956373108085
54 population size:  9915
55	4986	0.9912427027926303	0.49937958566813234	0.0004947791559225745	4921	0.9908890384896529	0.500201813194194	0.00018775068278036045
55 population size:  9907
56	4996	0.991256073891593	0.49939454295357205	0.0004940876208039042	4919	0.9908928561449737	0.5002084562500179	0.00018787382139375073
56 population size:  9915
57	5000	0.99126512

103	5069	0.991541637474782	0.49901676311068843	0.0005608530343155204	4853	0.9910358307298962	0.5001476646032997	0.00018412002776502335
103 population size:  9922
104	5062	0.9915453191739972	0.49899812061655646	0.000562004387412347	4850	0.9910369104800665	0.5001317274707491	0.00018407836794179155
104 population size:  9912
105	5057	0.9915469572825426	0.49898059854572985	0.0005622874624817436	4864	0.991033208223277	0.5001298704733422	0.00018379472264751164
105 population size:  9921


KeyboardInterrupt: 